#### Import Necessari

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import random
from sklearn.multiclass import OneVsOneClassifier

#### Caricamento Dataset

In [50]:
train1 = pd.read_json('dataset/Dataset20-21.json');
train2 = pd.read_json('dataset/Dataset21-22.json');
trainSet = pd.concat([train1, train2], ignore_index=True)
validationSet = pd.read_json('dataset/Dataset22-23.json');
testSet = pd.read_json('dataset/Dataset23-24.json');
trainSet.loc[trainSet["risultato"] == "X", "risultato"] = 3
trainSet.loc[trainSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "X", "risultato"] = 3

In [51]:
X_train = trainSet.drop("risultato", axis=1)
y_train = trainSet["risultato"]

X_validation = validationSet.drop("risultato", axis=1)
y_validation = validationSet["risultato"]

X_test = testSet.drop("risultato", axis=1)
y_test = testSet["risultato"]


In [52]:
X_train_noName = X_train
X_train_noName = X_train_noName.drop("squadraa", axis=1)
X_train_noName = X_train_noName.drop("squadrab", axis=1)
X_train_noName = X_train_noName.astype(float)
y_train=y_train.astype(float)

X_validation_noName = X_validation
X_validation_noName = X_validation_noName.drop("squadraa", axis=1)
X_validation_noName = X_validation_noName.drop("squadrab", axis=1)
X_validation_noName = X_validation_noName.astype(float)
y_validation=y_validation.astype(float)

X_test_noName = X_test
X_test_noName = X_test_noName.drop("squadraa", axis=1)
X_test_noName = X_test_noName.drop("squadrab", axis=1)
X_test_noName = X_test_noName.astype(float)



#### Inizio Codice training

In [53]:
def evaluate_model(C, kernel, X_train, y_train, X_validation, y_validation):
    svm_model = SVC(C=C, kernel=kernel)
    svm_model = OneVsOneClassifier(svm_model)
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_validation)
    accuracy = accuracy_score(y_validation, y_pred)
    return accuracy

best_C = 1.0
best_kernel = 'linear'
best_accuracy = 0.0
max_iterations = 400

mask = ~np.isnan(y_validation)
y_validation = y_validation[mask]
X_validation_noName = X_validation_noName[mask] #Rimozione righe con NaN Values


for _ in range(max_iterations):
    new_C = random.uniform(0.1, 10.0)
    new_kernel = random.choice(['linear', 'rbf', 'poly'])
    accuracy = evaluate_model(new_C, new_kernel, X_train_noName, y_train, X_validation_noName, y_validation)
    if accuracy > best_accuracy:
        best_C = new_C
        best_kernel = new_kernel
        best_accuracy = accuracy

final_model = SVC(C=best_C, kernel=best_kernel)
final_model.fit(X_train_noName, y_train)
y_test_pred = final_model.predict(X_test_noName)
print("Best Hyperparameters: C =", best_C, ", Kernel =", best_kernel, ", Accuracy= ", best_accuracy)

Best Hyperparameters: C = 1.0913863053091735 , Kernel = linear , Accuracy=  0.5382585751978892


In [54]:
y_test_pred=y_test_pred.astype(int)
testSet['risultato']=y_test_pred
#testSet.to_csv('dataset/results.csv')